In [ ]:
%%cu
#include <stdio.h>

#define N 16 // rows
#define M 16 // columns
#define BLOCK_SIZE 4 // threads per block

__global__ void sumMatrixRows(int* matrix, int* result) {
    __shared__ int parcijalneSume[BLOCK_SIZE];

    // Initialize partial sum for this thread
    int parcijalSum = 0;
    int tid=threadIdx.x;

    parcijalSum += matrix[ blockIdx.x *2*  blockDim.x + threadIdx.x] + matrix [ blockIdx.x * 2 *  blockDim.x + threadIdx.x + BLOCK_SIZE];

    parcijalneSume[threadIdx.x] = parcijalSum;
    __syncthreads();

    for (int stride = blockDim.x/2; stride > 0; stride >>= 1) {
        if (threadIdx.x < stride) {
            parcijalneSume[threadIdx.x] += parcijalneSume[threadIdx.x + stride];
        }
        __syncthreads();
    }

    if (tid == 0) {
        result[ (blockIdx.x) ] += parcijalneSume[0];
    }
}

int main() {
    int matrix[N][M];
    int result[N];
    int* d_matrix;
    int* d_result;

    for (int i = 0; i < N; i++) {
        for (int j = 0; j < M; j++) {
            matrix[i][j] = i;
        }
    }

    cudaMalloc((void**)&d_matrix, N * M * sizeof(int));
    cudaMalloc((void**)&d_result, N * sizeof(int));

    cudaMemcpy(d_matrix, matrix, N * M * sizeof(int), cudaMemcpyHostToDevice);

    sumMatrixRows<<<N, N/2>>>(d_matrix, d_result);

    cudaMemcpy(result, d_result, N * sizeof(int), cudaMemcpyDeviceToHost);
    cudaFree(d_matrix);
    cudaFree(d_result);

    for (int i = 0; i < N; i++) {
        printf("Suma reda %d: %d\n", i, result[i]);
    }
    return 0;
}///POSLEDNJE MODIFIKACIJE NE RADE 28.8.

Suma reda 0: 0
Suma reda 1: 16
Suma reda 2: 32
Suma reda 3: 48
Suma reda 4: 64
Suma reda 5: 80
Suma reda 6: 96
Suma reda 7: 112
Suma reda 8: 128
Suma reda 9: 144
Suma reda 10: 160
Suma reda 11: 176
Suma reda 12: 192
Suma reda 13: 208
Suma reda 14: 224
Suma reda 15: 240



In [ ]:
%%cu
#include <stdio.h>

#define N 256 // rows
#define M 256 // columns
#define BLOCK_SIZE 8 // threads per block

__global__ void sumDiagonalOfMatrix(int* matrix, int* result) {
    __shared__ int parcijalneSume[BLOCK_SIZE];

    // Initialize partial sum for this thread
    int parcijalSum = 0;
    int tid=threadIdx.x;

    parcijalSum += matrix[ ( blockIdx.x * 2 *  blockDim.x + threadIdx.x ) * N + blockIdx.x * 2 *  blockDim.x + threadIdx.x ] +
    matrix [ ( blockIdx.x * 2 *  blockDim.x + threadIdx.x + BLOCK_SIZE ) * N + blockIdx.x * 2 *  blockDim.x + threadIdx.x + BLOCK_SIZE + 1 ];

    parcijalneSume[threadIdx.x] = parcijalSum;
    __syncthreads();

    for (int stride = blockDim.x/2; stride > 0; stride >>= 1) {
        if (threadIdx.x < stride) {
            parcijalneSume[threadIdx.x] += parcijalneSume[threadIdx.x + stride];
        }
        __syncthreads();
    }

    if (tid == 0) {
        result[blockIdx.x] = parcijalneSume[0];
    }
}

int main() {
    int matrix[N][M];
    int result[N];
    int* d_matrix;
    int* d_result;

    for (int i = 0; i < N; i++) {
        for (int j = 0; j < M; j++) {
            matrix[i][j] = 1;
        }
    }

    cudaMalloc((void**)&d_matrix, N * M * sizeof(int));
    cudaMalloc((void**)&d_result, N * sizeof(int));

    cudaMemcpy(d_matrix, matrix, N * M * sizeof(int), cudaMemcpyHostToDevice);

    sumDiagonalOfMatrix<<<N/BLOCK_SIZE/2, BLOCK_SIZE>>>(d_matrix, d_result);

    cudaMemcpy(result, d_result, N * sizeof(int), cudaMemcpyDeviceToHost);
    cudaFree(d_matrix);
    cudaFree(d_result);

    for (int i = 0; i < N / BLOCK_SIZE / 2; i++) {
        printf("Suma po blokovima JE %d\n", result[i]);
    }
    return 0;
}

Suma po blokovima JE 16
Suma po blokovima JE 16
Suma po blokovima JE 16
Suma po blokovima JE 16
Suma po blokovima JE 16
Suma po blokovima JE 16
Suma po blokovima JE 16
Suma po blokovima JE 16
Suma po blokovima JE 16
Suma po blokovima JE 16
Suma po blokovima JE 16
Suma po blokovima JE 16
Suma po blokovima JE 16
Suma po blokovima JE 16
Suma po blokovima JE 16
Suma po blokovima JE 15



In [ ]:
%%cu
#include <stdio.h>

#define N 256 // rows
#define M 256 // columns
#define BLOCK_SIZE 8 // threads per block

__global__ void sumDiagonalOfMatrix(int* matrix, int* result, int* globalSum) {
    __shared__ int parcijalneSume[BLOCK_SIZE];

    // Initialize partial sum for this thread
    int parcijalSum = 0;
    int tid = threadIdx.x;

    parcijalSum += matrix[(blockIdx.x * 2 * blockDim.x + tid) * N + blockIdx.x * 2 * blockDim.x + tid] +
                   matrix[(blockIdx.x * 2 * blockDim.x + tid + BLOCK_SIZE) * N + blockIdx.x * 2 * blockDim.x + tid + BLOCK_SIZE + 1];

    parcijalneSume[tid] = parcijalSum;
    __syncthreads();

    for (int stride = blockDim.x / 2; stride > 0; stride >>= 1) {
        if (tid < stride) {
            parcijalneSume[tid] += parcijalneSume[tid + stride];
        }
        __syncthreads();
    }

    if (tid == 0) {
        result[blockIdx.x] = parcijalneSume[0];
    }

    // Use atomicAdd to update the globalSum with the block's result
    if (tid == 0) {
        atomicAdd(globalSum, result[blockIdx.x]);
    }
}

int main() {
    int matrix[N][M];
    int result[N];
    int* d_matrix;
    int* d_result;
    int* d_globalSum;
    int globalSum = 0;

    for (int i = 0; i < N; i++) {
        for (int j = 0; j < M; j++) {
            matrix[i][j] = 1;
        }
    }

    cudaMalloc((void**)&d_matrix, N * M * sizeof(int));
    cudaMalloc((void**)&d_result, N * sizeof(int));
    cudaMalloc((void**)&d_globalSum, sizeof(int));

    cudaMemcpy(d_matrix, matrix, N * M * sizeof(int), cudaMemcpyHostToDevice);

    // Initialize globalSum on the device to 0
    cudaMemset(d_globalSum, 0, sizeof(int));

    sumDiagonalOfMatrix<<<N / BLOCK_SIZE / 2, BLOCK_SIZE>>>(d_matrix, d_result, d_globalSum);

    cudaMemcpy(result, d_result, N * sizeof(int), cudaMemcpyDeviceToHost);
    cudaMemcpy(&globalSum, d_globalSum, sizeof(int), cudaMemcpyDeviceToHost);

    cudaFree(d_matrix);
    cudaFree(d_result);
    cudaFree(d_globalSum);

    for (int i = 0; i < N / BLOCK_SIZE / 2; i++) {
        printf("Suma po blokovima JE %d\n", result[i]);
    }

    printf("Global sum of all block sums: %d\n", globalSum);

    return 0;
}


In [ ]:
!nvcc --version
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin


nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0
  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-cg6fihsh
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-cg6fihsh
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 0a71d56e5dce3ff1f0dd2c47c29367629262f527
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4295 sha256=d2d2dd57bb9ce2436894e1c5e99896214bb872683aadee6988e9b5bfb71889ed
  Stored in directory: /tmp/pip-ephem-wheel-cache-kuc55_eb/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built NVCCPlugin
created output directory at /content/src
Out bin /content/result.out
